<a href="https://colab.research.google.com/github/CE334/CE334_230333_Chinthala_Manasa/blob/main/CE334_230333_Chinthala_Manasa_Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
# Path to your CG-6 file
file_path = "/content/CG-6_0251_G1_CE335.dat"


# Step 1: Identify where actual data begins


with open(file_path, "r") as f:
    lines = f.readlines()

for i, line in enumerate(lines):
    if line.startswith("/Station"):
        header_index = i
        break

# Reading the data into a DataFrame


df = pd.read_csv(
    file_path,
    sep="\t",              # CG-6 files are tab-separated
    skiprows=header_index,
    engine="python"
)

# Cleaning column names
df.columns = df.columns.str.replace("/", "").str.strip()

# Step 3: Extract Required Columns


stations = df["Station"]
date = df["Date"]
time = df["Time"]
gravity = df["CorrGrav"]
std_dev = df["StdDev"]

# Displaying first few rows to verify
print(df.head())


  Station        Date      Time   CorrGrav  Line  StdDev  StdErr    RawGrav  \
0     FF1  2026-01-29  10:20:06  2911.7204     0  0.0135  0.0025  2919.8802   
1     FF1  2026-01-29  10:20:36  2911.7149     0  0.0116  0.0021  2919.8747   
2     FF1  2026-01-29  10:21:06  2911.7115     0  0.0088  0.0016  2919.8710   
3     TF1  2026-01-29  10:28:46  2909.6447     0  0.0165  0.0030  2917.8020   
4     TF1  2026-01-29  10:29:16  2909.6437     0  0.0122  0.0022  2917.8010   

     X    Y  ...  DriftCorr  MeasurDur  InstrHeight    LatUser    LonUser  \
0  3.6 -1.2  ...     -7.811         30          0.0  26.515875  80.230408   
1  3.8 -2.0  ...     -7.811         30          0.0  26.515875  80.230408   
2  4.8 -1.5  ...     -7.811         30          0.0  26.515875  80.230408   
3 -1.9 -4.1  ...     -7.811         30          0.0  26.515963  80.230484   
4 -0.7 -4.5  ...     -7.811         30          0.0  26.515963  80.230484   

   ElevUser     LatGPS     LonGPS  ElevGPS  \
0     105.7  26.

In [22]:
import numpy as np
import pandas as pd

# SETUP: Ensure we have the start time and the subset of data

# Convert 'Date' and 'Time' columns to datetime objects
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

# Calculate time in seconds from midnight for all entries
df['Time (Seconds from Midnight)'] = df['DateTime'].dt.hour * 3600 + \
                                     df['DateTime'].dt.minute * 60 + \
                                     df['DateTime'].dt.second

# The first row is our FIXED reference (FF1 at t1)
t1 = df.iloc[0]['Time (Seconds from Midnight)']
g_ref = df.iloc[0]['CorrGrav'] # Use the correct column name 'CorrGrav'

# We create equations for all SUBSEQUENT rows (Indices 1 to N)
observations = df.iloc[1:].copy().reset_index(drop=True) # Use .copy() to avoid SettingWithCopyWarning
num_equations = len(observations)

# DEFINE THE UNKNOWNS
# We map each point to a specific column index (0, 1, or 2) - now mapping station prefixes
col_map = {
    'TF': 0,
    '5F': 1,
    'SF': 2
}

# INITIALIZE MATRICES
# A is (num_equations)x4 (Equations, 4 Unknowns: g_TF, g_5F, g_SF, Drift)
A = np.zeros((num_equations, 4))
# L is (num_equations)x1 (The observed gravity differences)
L = np.zeros((num_equations, 1))

# FILL THE MATRIX (Row by Row)
for i, row in observations.iterrows():
    station_prefix = row['Station'][:2] # Extract 'TF', '5F', 'SF'
    current_time = row['Time (Seconds from Midnight)']
    observed_g = row['CorrGrav']

    # FILL COLUMNS 0-2 (Gravity Unknowns)
    if station_prefix in col_map:
        col_index = col_map[station_prefix]
        A[i, col_index] = 1.0

    # FILL COLUMN 3 (Drift Term)
    A[i, 3] = current_time - t1

    # FILL L VECTOR
    L[i, 0] = observed_g - g_ref

# DISPLAY THE RESULT
print("--- Design Matrix A ({}x4) ---".format(num_equations))
print("Cols: [g_TF, g_5F, g_SF, Drift]")
# Setting print options to suppress scientific notation for clarity
np.set_printoptions(suppress=True, precision=4)
print(A)

print("\n Observation Vector L ")
print(L)

--- Design Matrix A (29x4) ---
Cols: [g_TF, g_5F, g_SF, Drift]
[[   0.    0.    0.   30.]
 [   0.    0.    0.   60.]
 [   1.    0.    0.  520.]
 [   1.    0.    0.  550.]
 [   1.    0.    0.  580.]
 [   0.    0.    0.  887.]
 [   0.    0.    0.  917.]
 [   0.    0.    0.  947.]
 [   1.    0.    0. 1255.]
 [   1.    0.    0. 1285.]
 [   1.    0.    0. 1315.]
 [   0.    1.    0. 1597.]
 [   0.    1.    0. 1627.]
 [   0.    1.    0. 1657.]
 [   1.    0.    0. 1904.]
 [   1.    0.    0. 1934.]
 [   1.    0.    0. 1964.]
 [   0.    1.    0. 2219.]
 [   0.    1.    0. 2249.]
 [   0.    1.    0. 2279.]
 [   0.    0.    1. 2525.]
 [   0.    0.    1. 2555.]
 [   0.    0.    1. 2585.]
 [   0.    1.    0. 2898.]
 [   0.    1.    0. 2928.]
 [   0.    1.    0. 2958.]
 [   0.    0.    1. 3217.]
 [   0.    0.    1. 3247.]
 [   0.    0.    1. 3277.]]

 Observation Vector L 
[[-0.0055]
 [-0.0089]
 [-2.0757]
 [-2.0767]
 [-2.0797]
 [-0.0125]
 [-0.0132]
 [-0.0126]
 [-2.0765]
 [-2.0755]
 [-2.0767]
 [-4.267

In [23]:
import numpy as np

#  GET REFERENCE STD (Sigma_ref from the first row 'FF1')
sigma_ref = df.iloc[0]['StdDev'] # Corrected: used df and 'StdDev' column

# GET OBSERVATION STDS (Sigma_i for the remaining rows)
sigma_obs = observations['StdDev'].values # Corrected: used observations and 'StdDev' column

# CALCULATE WEIGHTS
# The variance of the difference (L_i - L_ref) is (sigma_i^2 + sigma_ref^2)
# Weight P = 1 / Variance
variances = (sigma_obs ** 2) + (sigma_ref ** 2)
weights = 1 / variances

#  CONSTRUCT THE DIAGONAL MATRIX (num_equations x num_equations)
P = np.diag(weights)

#  PRINT THE RESULTS
print(f"Reference StdDev (FF1): {sigma_ref:.6f}")
print(f"Number of Weights: {len(weights)}")
print("\n--- Weight Matrix P ({}x{}) ---".format(len(weights), len(weights))) # Dynamic size
# Set print options to fit the matrix on screen nicely
np.set_printoptions(linewidth=150, precision=2, suppress=True)
print(P)

Reference StdDev (FF1): 0.013500
Number of Weights: 29

--- Weight Matrix P (29x29) ---
[[3156.47    0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.
     0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.  ]
 [   0.   3850.75    0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.
     0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.  ]
 [   0.      0.   2200.22    0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.
     0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.  ]
 [   0.      0.      0.   3020.33    0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.
     0.      0.      0.      0.      0.      0.      0.      0.

In [24]:
import numpy as np
import sys

#  CALCULATE NORMAL MATRIX (N)
N = A.T @ P @ A

# CALCULATE RIGHT HAND SIDE VECTOR (U)
U = A.T @ P @ L

# INVERT THE NORMAL MATRIX
N_inv = np.linalg.inv(N)

#  SOLVE FOR X
X = N_inv @ U

#  PRINT THE EXACT RESULTS
# Set NumPy to print as many digits as needed to uniquely identify the number
np.set_printoptions(formatter={'float': '{: 0.20f}'.format}, suppress=False)

print(" Solution Vector X (4x1) [Full Precision] ")
print(X)

print("\n Interpreted Results (Relative to FF1) ")
# Using .25f ensures we see down to the noise floor of a 64-bit float
print(f"Gravity Diff (TF - FF): {X[0,0]:.25f} mGal")
print(f"Gravity Diff (5F - FF): {X[1,0]:.25f} mGal")
print(f"Gravity Diff (SF - FF): {X[2,0]:.25f} mGal")

# Printing drift in scientific notation is best for very small numbers to avoid seeing "0.0000..."
print(f"Instrument Drift Rate:  {X[3,0]:.25e} mGal/sec")

# Convert Drift to mGal/hour
drift_per_hour = X[3,0] * 3600
print(f"Drift Rate (per hour):  {drift_per_hour:.25f} mGal/hr")

 Solution Vector X (4x1) [Full Precision] 
[[-2.07281262808976762813]
 [-4.26140784175830233949]
 [-6.42878535905850867493]
 [-0.00000285312957554185]]

 Interpreted Results (Relative to FF1) 
Gravity Diff (TF - FF): -2.0728126280897676281256281 mGal
Gravity Diff (5F - FF): -4.2614078417583023394854536 mGal
Gravity Diff (SF - FF): -6.4287853590585086749342736 mGal
Instrument Drift Rate:  -2.8531295755418462078978337e-06 mGal/sec
Drift Rate (per hour):  -0.0102712664719506463484322 mGal/hr


In [25]:
import numpy as np

# CALCULATE RESIDUALS (V)
# V = A * X - L
V = (A @ X) - L

# PRINT RESIDUALS WITHOUT ROUNDING
# Set print options to show full precision (up to 20 decimal places)
np.set_printoptions(formatter={'float': '{: 0.20f}'.format}, suppress=False)

print(" Residual Vector V (9x1) [mGal] ")
print(V)

# Optional: Print them individually for clarity
print("\nIndividual Residuals")
for i, val in enumerate(V):
    print(f"v_{i+1}: {val[0]:.20f}")

 Residual Vector V (9x1) [mGal] 
[[ 0.00541440611294474738]
 [ 0.00872881222573451687]
 [ 0.00140374453127511956]
 [ 0.00231815064375773261]
 [ 0.00523255675664824338]
 [ 0.00996927406676723082]
 [ 0.01058368017955263474]
 [ 0.00989808629198224432]
 [ 0.00010669429305121270]
 [-0.00097889959396413317]
 [ 0.00013550651846827577]
 [ 0.00183571030963669557]
 [ 0.00155011642242097736]
 [ 0.00336452253535934886]
 [-0.00314498680157893062]
 [-0.00203058068869177433]
 [-0.00471617457576023469]
 [-0.00123893628625104668]
 [-0.00062453017346619788]
 [-0.00051012406078232431]
 [ 0.00321048876350360501]
 [ 0.00442489487613872967]
 [-0.00186069901093599555]
 [-0.00007621126814072454]
 [-0.00026180515515417113]
 [-0.00444739904252777762]
 [-0.00246387690297922290]
 [-0.00194947078994189837]
 [-0.00293506467720927589]]

Individual Residuals
v_1: 0.00541440611294474738
v_2: 0.00872881222573451687
v_3: 0.00140374453127511956
v_4: 0.00231815064375773261
v_5: 0.00523255675664824338
v_6: 0.00996927406676

In [26]:
import numpy as np

# Define the threshold for "Almost Zero"
THRESHOLD = 1e-6

# PRE-CALCULATE COMMON TERMS
# Weighted Sum of Squared Residuals (Scalar)
VTPV = V.T @ P @ V

# 2. Weighted Sum of Squared Observations (Scalar)
LTPL = L.T @ P @ L

# 3. U Vector (A_transpose * P * L)
ATPL = A.T @ P @ L   # This is U
U = ATPL

# 4. Term (L_transpose * P * A) - This is just U_transpose
LTPA = L.T @ P @ A

# 5. Inverse Normal Matrix
N_inv = np.linalg.inv(A.T @ P @ A)


print("="*40)
print(f"QUALITY CHECKS (Threshold: {THRESHOLD})")
print("="*40)


# CHECK 1
# Formula: V^T P V  ==  -L^T P A (A^T P A)^-1 A^T P L + L^T P L

# Calculate RHS: -(L^T P A) * (N_inv) * (A^T P L) + (L^T P L)
Check1_RHS = - (LTPA @ N_inv @ ATPL) + LTPL

print("\n--- CHECK 1 ---")
print(f"LHS (V^T P V): {VTPV[0,0]:.10f}")
print(f"RHS (Formula): {Check1_RHS[0,0]:.10f}")

diff1 = abs(VTPV - Check1_RHS)[0,0]
if diff1 < THRESHOLD:
    print(f"Result:  PASSED (Difference: {diff1:.10e})")
else:
    print(f"Result:  FAILED (Difference: {diff1:.10e})")



# CHECK 2
# Formula: A^T P V ≈ 0

Check2_Val = A.T @ P @ V

print("\n--- CHECK 2 ---")
print("Vector A^T P V (Should be close to zero):")
# Flatten for cleaner printing
print(Check2_Val.flatten())

# Check if ALL elements are below threshold
max_error_2 = np.max(np.abs(Check2_Val))
if max_error_2 < THRESHOLD:
    print(f"Result:  PASSED (Max Deviation: {max_error_2:.10e})")
else:
    print(f"Result:  FAILED (Max Deviation: {max_error_2:.10e})")



# CHECK 3
# Formula: V^T P V == -(A^T P L)^T X + L^T P L

# Note: (A^T P L)^T is U_transpose
Check3_RHS = - (U.T @ X) + LTPL

print("\n--- CHECK 3 ---")
print(f"LHS (V^T P V): {VTPV[0,0]:.10f}")
print(f"RHS (Formula): {Check3_RHS[0,0]:.10f}")

diff3 = abs(VTPV - Check3_RHS)[0,0]
if diff3 < THRESHOLD:
    print(f"Result:  PASSED (Difference: {diff3:.10e})")
else:
    print(f"Result:  FAILED (Difference: {diff3:.10e})")

QUALITY CHECKS (Threshold: 1e-06)

--- CHECK 1 ---
LHS (V^T P V): 1.6563612316
RHS (Formula): 1.6563612316
Result:  PASSED (Difference: 4.9980020123e-12)

--- CHECK 2 ---
Vector A^T P V (Should be close to zero):
[-0.00000000011468870298  0.00000000029553026692 -0.00000000010974136161  0.00000033343994480651]
Result:  PASSED (Max Deviation: 3.3343994481e-07)

--- CHECK 3 ---
LHS (V^T P V): 1.6563612316
RHS (Formula): 1.6563612320
Result:  PASSED (Difference: 4.6066328530e-10)
